# Libraries

In [35]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing

# Dataset Loading and Pre-Processing


In [36]:
# TODO : Copy Dataframes here
with open('C:/Users/Kaushal Kavuri/Downloads/TravelPackageRecommender-main (1)/TravelPackageRecommender-main/travel_cb.txt') as f:
    lines = f.readlines()

for i in range(len(lines)):
    lines[i]=lines[i].strip()
    
count = 0
dic = {}
inner = {}
for i in range(len(lines)):
    dic[count] = {}
    pos = lines[i].find(":")
    if(pos != -1):
        attr = lines[i][:pos]
        val = lines[i][pos+2:len(lines[i])-1].strip()
        inner[attr] = val
        if(attr == "Hotel"):
            dic[count] = inner
            inner = {}
            count += 1
print(dic[10])

{'JourneyCode': '11', 'HolidayType': 'City', 'Price': '1978', 'NumberOfPersons': '2', 'Region': 'Cairo', 'Transportation': 'Plane', 'Duration': '7', 'Season': 'April', 'Accommodation': 'ThreeStars', 'Hotel': '"Hotel Victoria, Cairo"'}


In [37]:
main=pd.DataFrame.from_dict(dic,orient="index")
N = 3
# Drop last N columns of dataframe
df = main.iloc[: , :-N]
# drop rows with NaN as values
df = df.dropna()
# add feature - weight
df['Weight'] = 1
input_df = df
df

,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,Bathing,2498,2,Egypt,Plane,14,April,TwoStars,"""Hotel White House, Egypt""",1
1,2,Bathing,3066,3,Egypt,Plane,21,May,TwoStars,"""Hotel White House, Egypt""",1
2,3,Active,2778,2,Egypt,Plane,14,June,TwoStars,"""Hotel Geisum, Egypt""",1
3,4,Bathing,3195,3,Egypt,Plane,7,July,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
4,5,Bathing,3158,2,Egypt,Plane,14,September,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
...,...,...,...,...,...,...,...,...,...,...,...
1471,1443,Bathing,1518,2,TurkishRiviera,Plane,14,December,ThreeStars,"""Hotel Santana, Turk. Riviera""",1
1473,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1475,1466,Recreation,1765,3,Tunisia,Plane,7,January,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1477,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1479
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   JourneyCode      1475 non-null   object
 1   HolidayType      1475 non-null   object
 2   Price            1475 non-null   object
 3   NumberOfPersons  1475 non-null   object
 4   Region           1475 non-null   object
 5   Transportation   1475 non-null   object
 6   Duration         1475 non-null   object
 7   Season           1475 non-null   object
 8   Accommodation    1475 non-null   object
 9   Hotel            1475 non-null   object
 10  Weight           1475 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 138.3+ KB


In [39]:
#TODO: Normalize data into numerical values
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
# https://stackoverflow.com/questions/42196589/any-way-to-get-mappings-of-a-label-encoder-in-python-pandas
df['Transportation']=le.fit_transform(df['Transportation'])
le.fit(df['Transportation'])
le_transportation_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Transportation']=le.transform(df['Transportation'])
le.fit(df['Season'])
le_season_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Season']=le.transform(df['Season'])
df['Season']

0        0
1        8
2        6
3        5
4       11
        ..
1471     2
1473     9
1475     4
1477     9
1479     4
Name: Season, Length: 1475, dtype: int32

In [40]:
le.fit(df['HolidayType'])
le_holiday_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['HolidayType']=le.transform(df['HolidayType'])

le.fit(df['Region'])
le_region_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Region']=le.transform(df['Region'])

le.fit(df['Accommodation'])
le_acc_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Accommodation']=le.transform(df['Accommodation'])

le.fit(df['Hotel'])
le_hotel_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Hotel']=le.transform(df['Hotel'])
df.head()


,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,1,2498,2,27,2,14,0,5,413,1
1,2,1,3066,3,27,2,21,8,5,413,1
2,3,0,2778,2,27,2,14,6,5,260,1
3,4,1,3195,3,27,2,7,5,4,187,1
4,5,1,3158,2,27,2,14,11,4,187,1


In [41]:
le_holiday_mapping

{'Active': 0,
 'Bathing': 1,
 'City': 2,
 'Education': 3,
 'Language': 4,
 'Recreation': 5,
 'Skiing': 6,
 'Wandering': 7}

## Data Split

In [42]:
# X = data without 'Price'
X = df.iloc[:,0:]
X = X.drop(df.columns[[2]], axis=1)  # df.columns is zero-based pd.Index
X = X.drop(df.columns[[0]], axis=1)
#X['JourneyCode'] = X['JourneyCode'].astype(str).astype(int)
X['NumberOfPersons'] = X['NumberOfPersons'].astype(str).astype(int)
X['Duration'] = X['Duration'].astype(str).astype(int)
X.info()
X = X.values

X.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1479
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   HolidayType      1475 non-null   int32
 1   NumberOfPersons  1475 non-null   int32
 2   Region           1475 non-null   int32
 3   Transportation   1475 non-null   int64
 4   Duration         1475 non-null   int32
 5   Season           1475 non-null   int32
 6   Accommodation    1475 non-null   int32
 7   Hotel            1475 non-null   int32
 8   Weight           1475 non-null   int64
dtypes: int32(7), int64(2)
memory usage: 74.9 KB


(1475, 9)

In [43]:
# target label = Price
Y = df['Price']
Y = Y.values
Y
Y.shape

(1475,)

# KNN Algorithm

In [44]:
class KNN:
    def __init__(self, K = 3):
        self.K = K
        
    def euclidean_distance(self,a,b,w = 1): # initial weight = 1
        return np.sqrt(np.sum(w*((a-b)**2)))
    
    def fit(self,X,y):
        self.X_train = X
        self.y_train = y
        
    def predict(self,samples):
        predicted_labels = [self._predict(sample) for sample in samples]
        return np.array(predicted_labels)
    
    def _predict(self,sample):
        # distance between sample and training points
        distances = [self.euclidean_distance(sample,x_train) for x_train in self.X_train]
        # get k-nearest data after sorting
        k_nearest_indices = np.argsort(distances)[:self.K]
        # get k-nearest data labels
        k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]
        # arrange frequency to get common class label
        
        most_common = Counter(k_nearest_labels).most_common(self.K)
        
        # extrac the labels in 1-d list
        predicted_labels = []
        for set in most_common:
            predicted_labels.append(set[0])
        return predicted_labels

In [49]:
# create a sample data
d = {'HolidayType': [0], 'NumberOfPersons' : [2], 'Region':[27], 'Transportation': [0],"Duration":[21],"Season":[6],"Accomodation":[4],"Hotel":[187],"Weight":[1] }
samples = pd.DataFrame(data=d)
samples = samples.values
samples.shape

(1, 9)

In [20]:
samples

array([[  0,   2,  27,   0,  21,   6,   4, 187,   1]], dtype=int64)

'3066'

In [25]:
# Test KNN functionality
obj = KNN()
obj.fit(X,Y)

predicted_labels = obj.predict(samples)
print(predicted_labels)

[['3318' '3158' '2198']]


In [12]:
input_df.loc[df['Price'] == '3158']

,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
4,5,1,3158,2,27,2,14,11,4,187,1
1336,1337,5,3158,2,34,2,14,3,4,16,1
1374,1375,1,3158,2,40,2,14,3,5,22,1


In [70]:
def generate_query():
    
    # KNN instance
    knn = KNN()
    knn.fit(X,Y)
    #knn.predict_labels({}) # pass response as a dictionary
    flag=1
    inner = {'HolidayType': [0], 'NumberOfPersons' : [2], 'Region':[27], 'Transportation': [0],"Duration":[21],"Season":[6],"Accomodation":[4],"Hotel":[187],"Weight":[1] }
    temp = pd.DataFrame(data=inner)
    temp = temp.values
    inner['Region']=le_region_mapping[input("Enter your destination region")]
    print("Select one out of three questions")

    while(flag):
        ques=["Duration of your stay", "Number of people for the stay", "Type of destination (city/countryside)", "Do you want a more expensive or cheaper accomodation", "Not satisfied with results?", "Need more options?"]        #Not satisfied with results will ask if the customer wants to start over or end the conversation &&& Need more options will display the next three results.
        #add more questions and print only first three questions at a time and display the results each time.
        print(knn.predict(temp))
        while(1):
            i=1
            for qu in ques:
                print(str(i)+") "+qu)
                i+=1
            slt=int(input("Select one: "))-1
            op=ques[slt]
            if(len(ques)>3):
                ques=ques[:slt]+ques[slt+1:]
            if(op=="Duration of your stay"):
                inner['Duration']=int(input("Enter your response1"))
                temp = pd.DataFrame(data=inner)
                temp = temp.values
                print(knn.predict(temp))
                continue
            if(op=="Number of people for the stay"):
                inner['NumberOfPersons']=int(input("Enter your response"))
                temp = pd.DataFrame(data=inner)
                temp = temp.values
                #calculate the price according to number of people using (total price)/(number of people) if the number of people in the retrived case is not the same
                print(knn.predict(temp))
                continue
            if(op=="Type of destination (city/countryside)"):
                inner['HolidayType']=int(le_holiday_mapping[input("Enter your response")])
                temp = pd.DataFrame(data=inner)
                temp = temp.values
                #same as above
                print(knn.predict(temp))
                continue
            if(op=="Do you want a more expensive or cheaper accomodation"):
                accprice=input("Enter 1 for better acc and 0 for cheaper acc")
                #go through the list of acc for cheaper or richer options and display 3
                continue
            if(op=="Not satisfied with results?"):
                if(int(input("Want to start over? 1 for yes 0 for no"))==1):
                    break
                else:
                    return
            if(op=="Need more options?"):
                #print next three options
                continue
            else:
                print("Enter correct option")



generate_query()

Enter your destination regionEgypt
Select one out of three questions
[['3318' '3158' '2198']]
1) Duration of your stay
2) Number of people for the stay
3) Type of destination (city/countryside)
4) Do you want a more expensive or cheaper accomodation
5) Not satisfied with results?
6) Need more options?
Select one: 1
Enter your response110
[['3195' '3318' '3158']]
1) Number of people for the stay
2) Type of destination (city/countryside)
3) Do you want a more expensive or cheaper accomodation
4) Not satisfied with results?
5) Need more options?


KeyboardInterrupt: Interrupted by user

In [ ]:
# get input, store in dict and pass in KNN
count=0
final={}